In [1]:
"""
    https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html
        - AG_NEWS 데이터셋으로 Text를 분류하는 모델 학습
"""

'\n    https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html\n        - AG_NEWS 데이터셋으로 Text를 분류하는 모델 학습\n'

In [2]:
"""
    학습을 위한 데이터를 준비하는 과정
       - Iterable Dataset 불러옴 (여기서 사용하는 AG_NEWS는 torch.utils.dataset과 다름)
       - 불러온 Data 기반으로Vocab 만듬
"""

import torch
from torchtext.datasets import AG_NEWS
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

# AG New 데이터 불러옴
# https://pytorch.org/text/stable/datasets.html#ag-news
# train: 120000 / test: 7600 / class 수 = 4
train_iter = AG_NEWS(root="./data/",split='train') 
print(type(train_iter))
# next(train_iter) # (class, 텍스트)
tokenizer = get_tokenizer('basic_english')

# generator - text를 tokenization하여 yield
def yield_tokens(data_iter):
    for _,text in data_iter:
        yield tokenizer(text)



# 전체 text를 tokenization 하며 vocab 구축 
vocab = build_vocab_from_iterator(yield_tokens(train_iter),specials=["<unk>"]) 
vocab.set_default_index(vocab["<unk>"])

train_iter = AG_NEWS(root="./data/",split='train') 
num_class = len(set([label for label,text in train_iter]))
print("vocab size = %d"%len(vocab))
print("num class = %d"%num_class)

print(vocab(["here","is"]))

<class 'torchtext.data.datasets_utils._RawTextIterableDataset'>
vocab size = 95811
num class = 4
[475, 21]


In [3]:
"""
    IterableDataset을 활용! -> user가 정의한 Iterator에 따라 data loading 순서가 정해짐
    DataLoader를 활용하여 학습을 위한 Batch 데이터 생성
"""
from torch.utils.data import DataLoader

train_iter = AG_NEWS(root="./data/",split='train') 

get_token_id_list = lambda x : vocab(tokenizer(x))
get_label_id = lambda x : x-1


def collate_batch(batch):
    """
        dataset을 전처리 하여 
        batch 형태의 training_example, label, offset을 Tensor로 만듬
        
        Arguments:
            batch (List(Tuple)) - (label,text) - dataset으로부터 얻어진 tuple들의 묶음
        Return:
            아래의 Return은 Cutomized collate_fn의 return 값으로 향후 dataloader로부터 iteration 마다 추출할 batch 단위의 값들임 
            
            text_list (1d Tensor) - [batch_size * (each token num)] batch를 구성하는 각 Token들의 id list
            label_list (1d Tensor) - [batch size] batch를 구성하는 label들의 id list
            offset (1d Tensor) - [batch size] text_list에서 batch를 구성하는 각 example들의 index들    
    """

    label_list, text_list, offsets = [], [],[0]
    for (_label, _text) in batch:
        label_list.append(get_label_id(_label))
        text_tensor = torch.tensor(get_token_id_list(_text),dtype=torch.int64)
        text_list.append(text_tensor)
        offsets.append(text_tensor.size(0)) 
    
    offsets=torch.tensor(offsets).cumsum(dim=0)
    label_list = torch.tensor(label_list) # [batch size * 1(label id)]
    text_list = torch.cat(text_list,dim=0) # [batch_size * (each token num)]
    
    return label_list, text_list, offsets
        
        
# print(next(train_iter))
dataloader=DataLoader(train_iter,batch_size=8,shuffle=False,collate_fn=collate_batch)
# dataloader=DataLoader(train_iter,batch_size=1,shuffle=False)

print(next(iter(dataloader)))

(tensor([2, 2, 2, 2, 2, 2, 2, 2]), tensor([  431,   425,     1,  1605, 14838,   113,    66,     2,   848,    13,
           27,    14,    27,    15, 50725,     3,   431,   374,    16,     9,
        67507,     6, 52258,     3,    42,  4009,   783,   325,     1, 15874,
         1072,   854,  1310,  4250,    13,    27,    14,    27,    15,   929,
          797,   320, 15874,    98,     3, 27657,    28,     5,  4459,    11,
          564, 52790,     8, 80617,  2125,     7,     2,   525,   241,     3,
           28,  3890, 82814,  6574,    10,   206,   359,     6,     2,   126,
            1,    58,     8,   347,  4582,   151,    16,   738,    13,    27,
           14,    27,    15,  2384,   452,    92,  2059, 27360,     2,   347,
            8,     2,   738,    11,   271,    42,   240, 51953,    38,     2,
          294,   126,   112,    85,   220,     2,  7856,     6, 40066, 15380,
            1,    70,  7376,    58,  1810,    29,   905,   537,  2846,    13,
           27,    14,    27, 

In [10]:
"""
    위의 예시는 정의된 AG_News를 활용했다면...
    일반적으로 Torch 사용시를 위해 Custom Dataset을 정의하고 DataLoader로 데이터를 불러와보자
    
    Custom Dataset 관련 - https://pytorch.org/tutorials/beginner/data_loading_tutorial.html 
    
    TextDataset -> 데이터 전처리 (tokenization, tensor만듬) / mapy-style dataset
    DataLoader의 collate_fn -> batch 단위의 Tensor 생성 
        maps style의 dataset을 사용했으므로 shuffling 등 가능
"""
from torch.utils.data import Dataset, DataLoader
import time


class TextDataset(Dataset):
    """
        custom dataset을 위해서 다음 3가지의 method를 구현해야함
        dataset은 크게 2종류로 만들 수 있음 (iterable style, map style)
        여기서는 map style로 만들어봄
    """
    def __init__(self, root,split, transform=None):
        
        train_iter = AG_NEWS(root=root,split=split) 
        self.train_data = [x for x in train_iter]
        self.transform = transform

        
    def __len__(self):
        return len(self.train_data)
    
    def __getitem__(self,idx):
        
#         print(idx)
#         print("\n")
        """ transform method를 통해 전처리 수행 후 tensor로 바꾸어서 Return """
        return self.transform(self.train_data[idx])

def transform(data):
    """
        tokenization 수행 후 tensor로 바꿈
    """
#     print("transform")
    get_token_id_list = vocab(tokenizer(data[1]))
    get_label_id = data[0]-1

    return torch.tensor(get_label_id), torch.tensor(get_token_id_list)
    

def collate_batch(batch):
    
    label_list, text_list, offsets = [], [],[0]

    for _label, _text in batch:
        label_list.append(_label)
        text_list.append(_text)
        offsets.append(_text.size(0))
    
    label_list=torch.tensor(label_list)
    text_list=torch.cat(text_list,0)
    offsets=torch.tensor(offsets[:-1]).cumsum(dim=0)
    
    return label_list, text_list, offsets
    
text_dataset=TextDataset(root="./data/",split='train',transform=transform) 

""" 데이터가 잘 생성됨을 확인 """
# dataloader=DataLoader(text_dataset, batch_size=2, shuffle=True, collate_fn=collate_batch, )
# next(iter(dataloader)) 

""" Multi processing 사용시 데이터 전처리 후 Data Loading이 빨라짐을 확인 """


dataloader=DataLoader(text_dataset, batch_size=2, shuffle=False, collate_fn=collate_batch, num_workers=0, pin_memory=False)


print(next(iter(dataloader)))





(tensor([2, 2]), tensor([  431,   425,     1,  1605, 14838,   113,    66,     2,   848,    13,
           27,    14,    27,    15, 50725,     3,   431,   374,    16,     9,
        67507,     6, 52258,     3,    42,  4009,   783,   325,     1, 15874,
         1072,   854,  1310,  4250,    13,    27,    14,    27,    15,   929,
          797,   320, 15874,    98,     3, 27657,    28,     5,  4459,    11,
          564, 52790,     8, 80617,  2125,     7,     2,   525,   241,     3,
           28,  3890, 82814,  6574,    10,   206,   359,     6,     2,   126,
            1]), tensor([ 0, 29]))


In [5]:
"""
    Text를 구성하는 token들의 mean에 Linear layer 하나 추가하여 분류하는 모델 생성
"""
from torch import nn

class TextClassificationModel(nn.Module):
    
    def __init__(self, vocab_size, emb_size, num_class):
        super(TextClassificationModel,self).__init__()
        # Embedding Layer -> Token들 id list를 받아 Embedding 값의 mean을 만듬
        self.embedding=nn.EmbeddingBag(vocab_size,emb_size,sparse=True) 
        self.fc = nn.Linear(emb_size,num_class)

    # 추후 정의 모델의 inference 시 사용되는 method
    def forward(self,text,offsets):
        embed=self.embedding(text,offsets)
#         print(embed)
        return self.fc(embed)




vocab_size = len(vocab)
emb_size = 64
model = TextClassificationModel(vocab_size, emb_size, num_class)

# 데데이이터  추추론 예시
dataloader=DataLoader(text_dataset, batch_size=2, shuffle=False, collate_fn=collate_batch)
label, text, offset = next(iter(dataloader))


model.eval()
model(text,offset)


tensor([[-0.0072, -0.2905, -0.1237,  0.1631],
        [ 0.0127, -0.1741, -0.0472,  0.1953]], grad_fn=<AddmmBackward>)

In [48]:
"""
    <전체 정리>
    
    최종 모델 학습 및 평가 코드
    - Train, Valid, Test 데이터 분리리하여 학습 및 평가
    - Loss, optimization, scheduler 정의하여 모델 학습
    
"""

# gpu 있을 경우 사용
device=torch.device("cuda" if torch.cuda.is_available() else "cpu") 

# hyper parameter 
EPOCHS = 10
LR = 5
BATCH_SIZE = 256


# 위에서 정의한 TextDataset을 활용하여 Custom dataset 불러옴
train_dataset=TextDataset(root="./data/",split="train",transform=transform) 
test_dataset=TextDataset(root="./data/",split="test",transform=transform) 
len_train = int(len(train_dataset)*0.9)
len_valid = len(train_dataset)-len_train

# 9:1 비율로 Train, Valid dataset 나눔
train_dataset, valid_dataset=torch.utils.data.random_split(train_dataset,[len_train,len_valid]) 

# 위에서 정의한 batch 만드는 custom collate_fn 사용 / train, valid, test를 batch형태로 iteration 하기 위한 dataloader 생성
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, collate_fn=collate_batch)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, collate_fn=collate_batch)
test_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, collate_fn=collate_batch)

# 모델 생성
vocab_size = len(vocab)
emb_size = 64
model = TextClassificationModel(vocab_size, emb_size, num_class).to(device)


# loss function, optimizer, scheduler 설정
criterion = torch.nn.CrossEntropyLoss() # multi class classification을 위한 Cross entopy loss
optimizer = torch.optim.SGD(model.parameters(),lr=LR) # SGD Optimizer
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 1.0, gamma=0.1) # step size의 epoch마다 gamma를 learning rate에 곱하여 줄이는 scheduler

def print_log(true_num, total_num):
    print("accuracy %.4f |"%(true_num/total_num))

def data_to_device(*data):
    return tuple(k.to(device) for k in data)

    
def train(data_loader):
    """ 모델 학습 """
    total_num, true_num, loss_val = 0,0,0
    
    model.train() # 학습 모드 / gradient 계산 + drop out, batch normalization 등 활성화
    for idx, (label, text, offsets) in enumerate(data_loader):
        
        label, text, offsets = data_to_device(label, text, offsets) # gpu로 데이터 옮김 (사실 이는 data loader의 collate_fn에서 구현해도됨)
        
        optimizer.zero_grad() # gradient를 초기화 (초기화 하지 않으면 계산한 gradient가 계속 쌓임)
        pred = model(text,offsets) # batch input에 대해 text classification 추론
        loss = criterion(pred,label) # loss 값을 구함

        loss.backward() # back propagation으로 gradient 구함 -> model의 prameter각각의 grad 변수에 이 값이 저장됨

        optimizer.step() # 계산한 gradient를 사용하여 parameter update
        
        # 예측이 맞은 수와 전체 batch의 데이터 수 구함
        true_num += (pred.argmax(1) == label).sum().item() 
        total_num += label.size(0)
        loss_val+=loss.item()

        
        # 10000 batch 마다 정확도 log 남김
        if idx!=0 and idx%100==0:
            print("batchs %d/%d "%(idx,len(data_loader)),end="")
            
            print("loss %.5f "%loss_val,end="")
            print_log(true_num, total_num)
            total_num, true_num,loss_val = 0,0,0

def eval(data_loader):
    
    total_num, true_num = 0,0
    model.eval() # 평가 모드 / gradient 계산 + drop out, batch normalization 등 비활성화
    for idx, (label, text, offsets) in enumerate(data_loader):
        
        label, text, offsets = data_to_device(label, text, offsets)
        pred = model(text,offsets)
        
        true_num += (pred.argmax(1) == label).sum().item()
        total_num += label.size(0)
        
    return true_num, total_num

valid_acc = 0

for epochs in range(1, EPOCHS+1):
    print("====================================")
    print("EPOCH ",epochs)
    
    train(train_dataloader)
    
    # 1 epoch 학습 끝날 때마다 validation dataset에 대해 평가 후 accuracy 출력
    print("validation ",end="")
    true_num, total_num=eval(valid_dataloader)
    
    print_log(true_num, total_num)
    
    # validation dataset 기준으로 정확도가 높아지면 learning rate를 줄임
    if valid_acc<true_num/total_num:
        valid_acc=true_num/total_num
        scheduler.step()
        print("learning rate decay")
        
    
    
# Test Dataset으로 최종 평가
print("test ",end="")
print_log(*eval(test_dataloader))


EPOCH  1
batchs 100/422 loss 118.94082 accuracy 0.4812 |
batchs 200/422 loss 93.76521 accuracy 0.6232 |
batchs 300/422 loss 79.33162 accuracy 0.6881 |
batchs 400/422 loss 70.54334 accuracy 0.7308 |
validation accuracy 0.7561 |
learning rate decay
EPOCH  2
batchs 100/422 loss 65.52635 accuracy 0.7566 |
batchs 200/422 loss 64.28022 accuracy 0.7586 |
batchs 300/422 loss 62.82645 accuracy 0.7632 |
batchs 400/422 loss 62.19183 accuracy 0.7701 |
validation accuracy 0.7655 |
learning rate decay
EPOCH  3
batchs 100/422 loss 63.35445 accuracy 0.7657 |
batchs 200/422 loss 62.63051 accuracy 0.7668 |
batchs 300/422 loss 61.69266 accuracy 0.7687 |
batchs 400/422 loss 61.48773 accuracy 0.7723 |
validation accuracy 0.7662 |
learning rate decay
EPOCH  4
batchs 100/422 loss 63.12752 accuracy 0.7665 |
batchs 200/422 loss 62.46681 accuracy 0.7672 |
batchs 300/422 loss 61.58450 accuracy 0.7690 |
batchs 400/422 loss 61.41804 accuracy 0.7730 |
validation accuracy 0.7662 |
EPOCH  5
batchs 100/422 loss 63.104

In [33]:
def a(*a):
    for k in a:
        print(k)
    
tuple(k for k in range(3))

(0, 1, 2)

139920566125264
139920566125264
139920566125264
